In [1]:
import pandas as pd
import numpy as np
from rdkit.Chem.Fragments import * 
from rdkit import Chem

In [2]:
!python -V

Python 3.8.13


In [4]:
#ames_df = pd.read_csv("example/mydata.csv", index_col=0)
# check cddd preprocessing to see how many smiles fail
#ames_df = pd.read_csv("/Users/delser/mass2smiles/retrain/all-HRMS-loss-header-validation-refine.tsv", index_col="id",sep="\t")
#ames_df = pd.read_csv("/Users/delser/mass2smiles/retrain/lipidblast-loss-header-train-refine.tsv", index_col="id",sep="\t")
#ames_df = pd.read_csv("/Users/delser/downloads/all_CFM_loss_header.tsv", index_col="id",sep="\t")
#ames_df = pd.read_csv("/Users/delser/mass2smiles/retrain/nist/all-HRMS-loss-header-train-refine.tsv", index_col="id",sep="\t")
data_df = pd.read_csv("/Users/delser/mass2smiles/retrain/nist/all_HRMS_train_16122022_cddd_refine.tsv",sep="\t")
#data_df = pd.read_csv("/Users/delser/mass2smiles/retrain/nist/all_HRMS_validation_16122022_cddd_refine.tsv",sep="\t")
data_df=data_df.dropna()
data_df

,spectrum_id,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed
0,CCMSLIB00005883221,[M+H]+,243.088000,"[91.054298, 95.049202, 103.054298, 104.0495, 1...","[0.04068640697166502, 0.01589638945223964, 0.2...","[26.007495000000006, 27.011003999999986, 43.00...","[0.09251948915755456, 0.5639426077381412, 0.05...",ZJTJUVIJVLLGSP-UHFFFAOYSA-N,Cc1cc2nc3[nH]c(=O)[nH]c(=O)c3nc2cc1C
1,NIST937042,[M-H2O+H]+,205.085900,"[63.0231, 65.0387, 77.0386, 79.0543, 89.0387, ...","[0.0221021021021021, 0.24380380380380381, 0.06...","[17.0223, 18.010100000000023, 18.0305000000000...","[0.025495495495495495, 1.0, 0.0601001001001001...",OHQFGTDAUDLCBG-UHFFFAOYSA-N,Cc1cccc(CC(CC(=O)O)C(=O)O)c1
2,spec_id=107587,[M+H]+,370.130000,"[106.065, 113.072, 121.088, 137.056, 137.062, ...","[0.842, 0.599, 0.97, 1.0, 1.0, 1.0, 0.119, 0.0...","[20.004999999999995, 29.548000000000002, 30.49...","[0.387, 0.069, 0.011, 0.066, 0.159, 0.033, 1.0...",VPRBWEMXTXPBEY-UHFFFAOYSA-N,Cc1c(C)c2ccc3c(c2oc1=O)CN(CCc1ccc(Cl)cc1)CO3
3,Eriodictyol-7-O-glucoside_plasma_1981,[M+H]+,451.123488,"[67.01862, 69.03341, 79.01767, 80.06145, 89.03...","[0.084, 0.027, 0.013, 0.021, 0.048, 0.019, 0.0...","[163.059708, 164.07297800000003, 221.074678, 2...","[0.015, 0.023, 0.014, 0.023, 0.021, 0.013, 0.0...",RAFHNDRXYHOLSH-RGHIGTIISA-N,O=C1CC(c2ccc(O)c(O)c2)Oc2cc(OC3OC(CO)C(O)C(O)C...
4,NIST297310,[M+H]+,303.231900,"[50.0151, 51.0228, 53.0385, 55.0177, 55.0541, ...","[0.061401401401401406, 0.2772972972972973, 0.2...","[18.01049999999998, 33.0342, 34.2534, 36.02109...","[1.0, 0.051301301301301304, 0.0665965965965966...",GCKMFJBGXUYNAG-MPRNQXESSA-N,CC12CCC(=O)C=C1CCC1C2CCC2(C)C1CCC2(C)O
...,...,...,...,...,...,...,...,...,...
83353,CCMSLIB00004704976,[M+H]+,285.112000,"[61.011341, 103.054558, 131.049133, 132.052399...","[0.036569120000000004, 0.020349529999999998, 0...","[92.15045200000003, 103.05926600000004, 104.06...","[0.01426239, 0.09765834999999999, 1.0, 0.03854...",IAFBOKYTDSDNHV-UHFFFAOYSA-N,COc1cc(OC)c2c(c1)OC(c1ccccc1)CC2=O
83354,CCMSLIB00004712186,[M+K]+,189.068000,"[51.940292, 55.018112, 55.05452, 56.049725, 56...","[0.11617364000000001, 0.02121181, 0.1648827899...","[29.116019000000023, 30.09127000000001, 30.122...","[0.063098, 0.013058350000000002, 0.0179634, 0....",MGSRCZKZVOBKFT-UHFFFAOYSA-N,Cc1ccc(C(C)C)c(O)c1
83355,CCMSLIB00010119259,[M+H]+,271.096000,"[79.054443, 81.033691, 96.020523, 103.053909, ...","[0.01896576750013369, 0.07313959617720532, 0.0...","[42.01059699999999, 102.03124199999999, 108.05...","[0.04077342072895897, 0.01816400430866546, 0.0...",ODLVGCCGMXGMGZ-RMKNXTFCSA-N,COc1ccccc1C=CC(=O)c1ccc(O)cc1O
83356,NIST256349,[M+H]+,178.053200,"[55.0179, 57.0208, 58.9949, 62.9898, 70.0287, ...","[0.20500500500500501, 0.28869869869869874, 0.0...","[17.026399999999995, 19.05709999999999, 35.037...","[0.055005005005005006, 1.0, 0.0183983983983983...",XUHLIQGRKRUKPH-ITZCMCNPSA-N,C=CCS(=O)CC(N)C(=O)O


In [5]:
smiles_exp=data_df[["smiles_preprocessed","spectrum_id"]]
smiles_exp

,smiles_preprocessed,spectrum_id
0,Cc1cc2nc3[nH]c(=O)[nH]c(=O)c3nc2cc1C,CCMSLIB00005883221
1,Cc1cccc(CC(CC(=O)O)C(=O)O)c1,NIST937042
2,Cc1c(C)c2ccc3c(c2oc1=O)CN(CCc1ccc(Cl)cc1)CO3,spec_id=107587
3,O=C1CC(c2ccc(O)c(O)c2)Oc2cc(OC3OC(CO)C(O)C(O)C...,Eriodictyol-7-O-glucoside_plasma_1981
4,CC12CCC(=O)C=C1CCC1C2CCC2(C)C1CCC2(C)O,NIST297310
...,...,...
83353,COc1cc(OC)c2c(c1)OC(c1ccccc1)CC2=O,CCMSLIB00004704976
83354,Cc1ccc(C(C)C)c(O)c1,CCMSLIB00004712186
83355,COc1ccccc1C=CC(=O)c1ccc(O)cc1O,CCMSLIB00010119259
83356,C=CCS(=O)CC(N)C(=O)O,NIST256349


In [6]:
smiles_exp.to_csv("/Users/delser/mass2smiles/retrain/all_HRMS_train_16122022_cddd_smiles.txt",index=False,header=False,sep=" ")

In [7]:
!java -jar "SugarRemovalUtility-jar-with-dependencies.jar" -i "all_HRMS_train_16122022_cddd_smiles.txt" -t "3" -remTerm "false"


Sugar Removal Service Application starting. Evaluating command line arguments...
All command line arguments valid. Executing application...

Trying to load file at /Users/delser/mass2smiles/retrain/all_HRMS_train_16122022_cddd_smiles.txt
Found SMILES file.
Results will be written to: /Users/delser/mass2smiles/retrain/deglycosylation_results.txt
Logging will be done in file at: /Users/delser/mass2smiles/retrain/Log.txt

Given settings for Sugar Removal Utility:
	Type of moieties to remove: circular AND linear sugars (3)
	Detect circular sugars only with O-glycosidic bond: false
	Remove only terminal sugar moieties: false
	Preservation mode setting: HEAVY_ATOM_COUNT (1)
	Preservation mode threshold: 5
	Detect circular sugars only with enough exocyclic oxygen atoms: true
	Exocyclic oxygen atoms to atoms in ring ratio threshold: 0.5
	Detect linear sugars in rings: false
	Linear sugar candidate minimum size: 4 carbon atoms
	Linear sugar candidate maximum size: 7 carbon atoms
	Detect linear

In [8]:
ScanList=[]
with open("/Users/delser/mass2smiles/retrain/deglycosylation_results.txt", 'r') as f:     
 counter=0
 for line in f:
        line=line.strip('\n')
        ListID=line.split(';')
        ScanList.append(ListID)

In [8]:
len(ScanList)

745

In [9]:
ScanList[0]

['Nr',
 'ID',
 'originalMoleculeSMILES',
 'deglycosylatedMoleculeSMILES',
 'hadOrHasSugars',
 'SugarMoietySMILES']

In [10]:
ScanList[22][3:]

['COC1=CC=C2C=C3C4=C(CCN3C(=O)C2=C1OC)C=C5C(=C4)OCO5', 'false']

In [11]:
ScanList[22]

['22',
 'spec_id=94556',
 'COC1=CC=C2C=C3C4=C(CCN3C(=O)C2=C1OC)C=C5C(=C4)OCO5',
 'COC1=CC=C2C=C3C4=C(CCN3C(=O)C2=C1OC)C=C5C(=C4)OCO5',
 'false']

In [9]:
max([len(i[3:]) for i in ScanList])

5

In [10]:
sugars=[i for i in ScanList if 'true' in i]

In [11]:
len(sugars)

6819

In [12]:
sugars=[i for i in sugars if not '[empty]' in i]

In [13]:
len(sugars)

6367

In [15]:
max([len(i[5:]) for i in sugars])

3

In [16]:
sugars[26]

['305',
 'isookanin-7-O-glucoside_plasma_1863',
 'C1=C(C=C(C(=C1)O)O)C2CC(=O)C3=CC=C(C(=C3O2)O)OC4C(C(C(C(CO)O4)O)O)O',
 'C1=C(C=C(C(=C1)O)O)C2CC(=O)C3=CC=C(C(=C3O2)O)O',
 'true',
 '[CH]1[CH][CH]O[CH][CH]1']

In [14]:
test=[i for i in sugars if 'true' == i[4]]
len(test)

6367

In [19]:
sugars[26][1]

'isookanin-7-O-glucoside_plasma_1863'

In [20]:
sugars[26][3]

'C1=C(C=C(C(=C1)O)O)C2CC(=O)C3=CC=C(C(=C3O2)O)O'

In [17]:
end=[[i[1].strip('"'),i[3],len(i[5:])] for i in sugars]

In [49]:
end[6814]

['CCMSLIB00004710931',
 'C=C1C(CC2C(=COC(C12)OC3C(C(C(C(CO)O3)O)O)OC(=O)C=CC4=CC=C(C=C4)O)C(=O)O)O',
 0]

In [18]:
df = pd.DataFrame(end, columns =['spectrum_id','smiles_preprocessed','num_of_sugars'])
df 

,spectrum_id,smiles_preprocessed,num_of_sugars
0,Eriodictyol-7-O-glucoside_plasma_1981,C1=C(C=C(C(=C1)O)O)C2CC(=O)C3=C(C=C(C=C3O)O)O2,1
1,CCMSLIB00004703056,COC1=CC=C(C=CC(=O)OC2C3C=COC(C3C4(CO)C2O4)O)C=C1O,1
2,NP-000004(11)_plasma_2705,C1=CC(=CC=C1C2=CC(=O)C3=C(C=C(C(=C3O2)C4C(C(C(...,1
3,spec_id=75828,C1=CC=C(C=C1)C2=CC(=O)C3=C(C=C(C=C3O)O)O2,2
4,CCMSLIB00004708523,CC1=C(C(=O)O)C(C)(C)CCC1,2
...,...,...,...
6362,CCMSLIB00004715997,COC1=C(C=C2C3=C1OC(=O)C4=C3C(=C(C(=C4)O)OC)OC2...,1
6363,CCMSLIB00004710931,C=C1C(CC2C(=COC(C12)O)C(=O)O)O.C1=C(C=CC(=C1)O...,1
6364,CCMSLIB00006673475,C(CC(=O)O)C1=CC(=CC(=C1)O)O,1
6365,CCMSLIB00004708972,CC1=CC=CC2=C1C(=CC(=O)O2)O,1


In [19]:
df['num_of_sugars'].max()

3

In [20]:
df['num_of_sugars'].min()

1

In [21]:
df=df.set_index('spectrum_id')
df

,smiles_preprocessed,num_of_sugars
spectrum_id,,
Eriodictyol-7-O-glucoside_plasma_1981,C1=C(C=C(C(=C1)O)O)C2CC(=O)C3=C(C=C(C=C3O)O)O2,1
CCMSLIB00004703056,COC1=CC=C(C=CC(=O)OC2C3C=COC(C3C4(CO)C2O4)O)C=C1O,1
NP-000004(11)_plasma_2705,C1=CC(=CC=C1C2=CC(=O)C3=C(C=C(C(=C3O2)C4C(C(C(...,1
spec_id=75828,C1=CC=C(C=C1)C2=CC(=O)C3=C(C=C(C=C3O)O)O2,2
CCMSLIB00004708523,CC1=C(C(=O)O)C(C)(C)CCC1,2
...,...,...
CCMSLIB00004715997,COC1=C(C=C2C3=C1OC(=O)C4=C3C(=C(C(=C4)O)OC)OC2...,1
CCMSLIB00004710931,C=C1C(CC2C(=COC(C12)O)C(=O)O)O.C1=C(C=CC(=C1)O...,1
CCMSLIB00006673475,C(CC(=O)O)C1=CC(=CC(=C1)O)O,1


In [22]:
df=df.drop(["smiles_preprocessed"], axis=1)
df

,num_of_sugars
spectrum_id,
Eriodictyol-7-O-glucoside_plasma_1981,1
CCMSLIB00004703056,1
NP-000004(11)_plasma_2705,1
spec_id=75828,2
CCMSLIB00004708523,2
...,...
CCMSLIB00004715997,1
CCMSLIB00004710931,1
CCMSLIB00006673475,1


In [23]:
data_df=data_df.set_index('spectrum_id')
data_df

,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed
spectrum_id,,,,,,,,
CCMSLIB00005883221,[M+H]+,243.088000,"[91.054298, 95.049202, 103.054298, 104.0495, 1...","[0.04068640697166502, 0.01589638945223964, 0.2...","[26.007495000000006, 27.011003999999986, 43.00...","[0.09251948915755456, 0.5639426077381412, 0.05...",ZJTJUVIJVLLGSP-UHFFFAOYSA-N,Cc1cc2nc3[nH]c(=O)[nH]c(=O)c3nc2cc1C
NIST937042,[M-H2O+H]+,205.085900,"[63.0231, 65.0387, 77.0386, 79.0543, 89.0387, ...","[0.0221021021021021, 0.24380380380380381, 0.06...","[17.0223, 18.010100000000023, 18.0305000000000...","[0.025495495495495495, 1.0, 0.0601001001001001...",OHQFGTDAUDLCBG-UHFFFAOYSA-N,Cc1cccc(CC(CC(=O)O)C(=O)O)c1
spec_id=107587,[M+H]+,370.130000,"[106.065, 113.072, 121.088, 137.056, 137.062, ...","[0.842, 0.599, 0.97, 1.0, 1.0, 1.0, 0.119, 0.0...","[20.004999999999995, 29.548000000000002, 30.49...","[0.387, 0.069, 0.011, 0.066, 0.159, 0.033, 1.0...",VPRBWEMXTXPBEY-UHFFFAOYSA-N,Cc1c(C)c2ccc3c(c2oc1=O)CN(CCc1ccc(Cl)cc1)CO3
Eriodictyol-7-O-glucoside_plasma_1981,[M+H]+,451.123488,"[67.01862, 69.03341, 79.01767, 80.06145, 89.03...","[0.084, 0.027, 0.013, 0.021, 0.048, 0.019, 0.0...","[163.059708, 164.07297800000003, 221.074678, 2...","[0.015, 0.023, 0.014, 0.023, 0.021, 0.013, 0.0...",RAFHNDRXYHOLSH-RGHIGTIISA-N,O=C1CC(c2ccc(O)c(O)c2)Oc2cc(OC3OC(CO)C(O)C(O)C...
NIST297310,[M+H]+,303.231900,"[50.0151, 51.0228, 53.0385, 55.0177, 55.0541, ...","[0.061401401401401406, 0.2772972972972973, 0.2...","[18.01049999999998, 33.0342, 34.2534, 36.02109...","[1.0, 0.051301301301301304, 0.0665965965965966...",GCKMFJBGXUYNAG-MPRNQXESSA-N,CC12CCC(=O)C=C1CCC1C2CCC2(C)C1CCC2(C)O
...,...,...,...,...,...,...,...,...
CCMSLIB00004704976,[M+H]+,285.112000,"[61.011341, 103.054558, 131.049133, 132.052399...","[0.036569120000000004, 0.020349529999999998, 0...","[92.15045200000003, 103.05926600000004, 104.06...","[0.01426239, 0.09765834999999999, 1.0, 0.03854...",IAFBOKYTDSDNHV-UHFFFAOYSA-N,COc1cc(OC)c2c(c1)OC(c1ccccc1)CC2=O
CCMSLIB00004712186,[M+K]+,189.068000,"[51.940292, 55.018112, 55.05452, 56.049725, 56...","[0.11617364000000001, 0.02121181, 0.1648827899...","[29.116019000000023, 30.09127000000001, 30.122...","[0.063098, 0.013058350000000002, 0.0179634, 0....",MGSRCZKZVOBKFT-UHFFFAOYSA-N,Cc1ccc(C(C)C)c(O)c1
CCMSLIB00010119259,[M+H]+,271.096000,"[79.054443, 81.033691, 96.020523, 103.053909, ...","[0.01896576750013369, 0.07313959617720532, 0.0...","[42.01059699999999, 102.03124199999999, 108.05...","[0.04077342072895897, 0.01816400430866546, 0.0...",ODLVGCCGMXGMGZ-RMKNXTFCSA-N,COc1ccccc1C=CC(=O)c1ccc(O)cc1O


In [24]:
data_df=data_df.join(df,how="outer")
data_df

,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,num_of_sugars
spectrum_id,,,,,,,,,
(+)-Chelidonine_plasma_1338,[M+H]+,354.133599,"[135.045, 159.04369, 163.04027, 173.0605, 179....","[0.053, 0.011, 0.027, 0.015, 0.011, 0.034, 0.1...","[18.014429199999995, 30.038839199999984, 31.04...","[0.02, 0.045, 0.166, 0.026, 0.146, 0.041, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,NaN
(+)-Chelidonine_plasma_1340,[M+H]+,354.133599,"[275.0733, 305.08203, 323.09119, 324.09433, 35...","[0.017, 0.022, 0.028, 0.01, 1.0, 0.029]","[30.03926919999998, 31.04240920000001, 49.0515...","[0.01, 0.028, 0.022, 0.017]",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,NaN
(+)-Chelidonine_plasma_1343,[M+H]+,354.133599,"[135.04521, 143.05431, 159.04745, 161.0602, 16...","[0.101, 0.011, 0.012, 0.035, 0.081, 0.013, 0.0...","[18.008909200000005, 31.031639199999972, 31.04...","[0.016, 0.02, 0.012, 0.013, 0.028, 0.023, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,NaN
(+)-Chelidonine_plasma_1346,[M+H]+,354.133599,"[103.05478, 118.06499, 131.04845, 135.04384, 1...","[0.016, 0.01, 0.027, 0.034, 0.01, 0.016, 0.01,...","[79.06267919999999, 106.06514919999998, 107.05...","[0.023, 0.02, 0.063, 0.065, 0.038, 0.014, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,NaN
(+)-Chelidonine_plasma_1349,[M+H]+,354.133599,"[135.03654, 135.04454, 159.04517, 163.03651, 1...","[0.01, 0.047, 0.011, 0.016, 0.017, 0.014, 0.11...","[18.00828919999998, 30.03291919999998, 30.0438...","[0.028, 0.025, 0.015, 0.148, 0.03, 0.126, 0.04...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,NaN
...,...,...,...,...,...,...,...,...,...
trans-pterostilbene_plasma_3538,[M+H]+,257.117221,"[55.01796, 77.03296, 77.03867, 79.02107, 79.05...","[0.021, 0.021, 0.046, 0.011, 0.046, 0.02, 0.01...","[15.022540799999973, 16.0322008, 17.0336207999...","[0.259, 0.209, 0.02, 0.017, 0.034, 0.038, 0.01...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,NaN
trans-pterostilbene_plasma_3540,[M+H]+,257.117221,"[65.03999, 75.02646, 77.02925, 77.03927, 78.04...","[0.057, 0.011, 0.011, 0.115, 0.013, 0.043, 0.0...","[31.055850799999973, 45.04376079999997, 46.038...","[0.026, 0.011, 0.012, 0.013, 0.037, 0.016, 0.0...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,NaN
trans-pterostilbene_plasma_3542,[M+H]+,257.117221,"[91.05325, 103.0554, 105.07047, 107.05009, 111...","[0.015, 0.01, 0.058, 0.021, 0.034, 0.02, 0.015...","[15.01973079999999, 31.012410799999998, 31.021...","[0.043, 0.016, 0.018, 0.046, 0.012, 0.03, 0.03...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,NaN


In [25]:
data_df['num_of_sugars'] = data_df['num_of_sugars'].fillna(0)
data_df

,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,num_of_sugars
spectrum_id,,,,,,,,,
(+)-Chelidonine_plasma_1338,[M+H]+,354.133599,"[135.045, 159.04369, 163.04027, 173.0605, 179....","[0.053, 0.011, 0.027, 0.015, 0.011, 0.034, 0.1...","[18.014429199999995, 30.038839199999984, 31.04...","[0.02, 0.045, 0.166, 0.026, 0.146, 0.041, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0
(+)-Chelidonine_plasma_1340,[M+H]+,354.133599,"[275.0733, 305.08203, 323.09119, 324.09433, 35...","[0.017, 0.022, 0.028, 0.01, 1.0, 0.029]","[30.03926919999998, 31.04240920000001, 49.0515...","[0.01, 0.028, 0.022, 0.017]",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0
(+)-Chelidonine_plasma_1343,[M+H]+,354.133599,"[135.04521, 143.05431, 159.04745, 161.0602, 16...","[0.101, 0.011, 0.012, 0.035, 0.081, 0.013, 0.0...","[18.008909200000005, 31.031639199999972, 31.04...","[0.016, 0.02, 0.012, 0.013, 0.028, 0.023, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0
(+)-Chelidonine_plasma_1346,[M+H]+,354.133599,"[103.05478, 118.06499, 131.04845, 135.04384, 1...","[0.016, 0.01, 0.027, 0.034, 0.01, 0.016, 0.01,...","[79.06267919999999, 106.06514919999998, 107.05...","[0.023, 0.02, 0.063, 0.065, 0.038, 0.014, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0
(+)-Chelidonine_plasma_1349,[M+H]+,354.133599,"[135.03654, 135.04454, 159.04517, 163.03651, 1...","[0.01, 0.047, 0.011, 0.016, 0.017, 0.014, 0.11...","[18.00828919999998, 30.03291919999998, 30.0438...","[0.028, 0.025, 0.015, 0.148, 0.03, 0.126, 0.04...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0
...,...,...,...,...,...,...,...,...,...
trans-pterostilbene_plasma_3538,[M+H]+,257.117221,"[55.01796, 77.03296, 77.03867, 79.02107, 79.05...","[0.021, 0.021, 0.046, 0.011, 0.046, 0.02, 0.01...","[15.022540799999973, 16.0322008, 17.0336207999...","[0.259, 0.209, 0.02, 0.017, 0.034, 0.038, 0.01...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0
trans-pterostilbene_plasma_3540,[M+H]+,257.117221,"[65.03999, 75.02646, 77.02925, 77.03927, 78.04...","[0.057, 0.011, 0.011, 0.115, 0.013, 0.043, 0.0...","[31.055850799999973, 45.04376079999997, 46.038...","[0.026, 0.011, 0.012, 0.013, 0.037, 0.016, 0.0...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0
trans-pterostilbene_plasma_3542,[M+H]+,257.117221,"[91.05325, 103.0554, 105.07047, 107.05009, 111...","[0.015, 0.01, 0.058, 0.021, 0.034, 0.02, 0.015...","[15.01973079999999, 31.012410799999998, 31.021...","[0.043, 0.016, 0.018, 0.046, 0.012, 0.03, 0.03...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0


fr_Al_COO(mol)
#Number of aliphatic carboxylic acids
fr_Al_OH(mol)
#Number of aliphatic hydroxyl groups

fr_Al_OH_noTert(mol)
#Number of aliphatic hydroxyl groups excluding tert-OH

fr_ArN(mol)
#Number of N functional groups attached to aromatics

fr_Ar_COO(mol)
#Number of Aromatic carboxylic acide

fr_Ar_N(mol)
#Number of aromatic nitrogens

fr_Ar_NH(mol)
#Number of aromatic amines

fr_Ar_OH(mol)
#Number of aromatic hydroxyl groups

fr_COO(mol)
#Number of carboxylic acids

fr_COO2(mol)
#Number of carboxylic acids

fr_C_O(mol)
#Number of carbonyl O

fr_C_O_noCOO(mol)
#Number of carbonyl O, excluding COOH

fr_C_S(mol)
#Number of thiocarbonyl

fr_Imine(mol)
#Number of Imines

fr_NH0(mol)
#Number of Tertiary amines

fr_NH1(mol)
#Number of Secondary amines

fr_NH2(mol)
#Number of Primary amines

fr_N_O(mol)
#Number of hydroxylamine groups

fr_Ndealkylation2(mol)
#Number of tert-alicyclic amines (no heteroatoms, not quinine-like bridged N)

fr_Nhpyrrole(mol)
#Number of H-pyrrole nitrogens

fr_SH(mol)
#Number of thiol groups

fr_aldehyde(mol)
#Number of aldehydes

fr_alkyl_carbamate(mol)
#Number of alkyl carbamates (subject to hydrolysis)

fr_alkyl_halide(mol)
#Number of alkyl halides

fr_allylic_oxid(mol)
#Number of allylic oxidation sites excluding steroid dienone

fr_amide(mol)
#Number of amides


fr_aniline(mol)
#Number of anilines

fr_aryl_methyl(mol)
#Number of aryl methyl sites for hydroxylation

fr_azo(mol)
#Number of azo groups


fr_benzene(mol)
#Number of benzene rings


fr_bicyclic(mol)
#Bicyclic


fr_dihydropyridine(mol)
#Number of dihydropyridines

fr_epoxide(mol)
#Number of epoxide rings

fr_ester(mol)
#Number of esters

fr_ether(mol)
#Number of ether oxygens (including phenoxy)

fr_furan(mol)
#Number of furan rings

fr_guanido(mol)
#Number of guanidine groups

fr_halogen(mol)
Number of halogens

fr_imidazole(mol)
#Number of imidazole rings

fr_isothiocyan(mol)
#Number of isothiocyanates

fr_ketone(mol)
#Number of ketones

fr_ketone_Topliss(mol)
#Number of ketones excluding diaryl, a,b-unsat. dienones, heteroatom on Calpha

fr_lactam(mol)
#Number of beta lactams

fr_lactone(mol)
#Number of cyclic esters (lactones)

fr_methoxy(mol)
#Number of methoxy groups -OCH3


fr_nitrile(mol)
#Number of nitriles

fr_nitro(mol)
#Number of nitro groups


fr_oxazole(mol)
#Number of oxazole rings

fr_para_hydroxylation(mol)
#Number of para-hydroxylation sites

fr_phenol(mol)
#Number of phenols

fr_phos_acid(mol)
#Number of phosphoric acid groups

fr_phos_ester(mol)
#Number of phosphoric ester groups

fr_piperdine(mol)
#Number of piperdine rings

fr_priamide(mol)
#Number of primary amides

fr_pyridine(mol)
#Number of pyridine rings

fr_quatN(mol)
#Number of quaternary nitrogens

fr_sulfide(mol)
#Number of thioether

fr_thiazole(mol)
#Number of thiazole rings

fr_unbrch_alkane(mol)
#Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)


In [26]:
def get_func_gr(mol):
    mol=Chem.MolFromSmiles(mol)
    gr=[fr_Al_COO(mol),
     #Number of aliphatic carboxylic acids
     fr_Al_OH(mol),
     #Number of aliphatic hydroxyl groups
     fr_Al_OH_noTert(mol),
     #Number of aliphatic hydroxyl groups excluding tert-OH
     fr_ArN(mol),
     #Number of N functional groups attached to aromatics
     fr_Ar_COO(mol),
     #Number of Aromatic carboxylic acide
     fr_Ar_N(mol),
     #Number of aromatic nitrogens
     fr_Ar_NH(mol),
     #Number of aromatic amines
     fr_Ar_OH(mol),
     #Number of aromatic hydroxyl groups
     fr_COO(mol),
     #Number of carboxylic acids
     fr_COO2(mol),
     #Number of carboxylic acids
     fr_C_O(mol),
     #Number of carbonyl O
     fr_C_O_noCOO(mol),
     #Number of carbonyl O, excluding COOH
     fr_C_S(mol),
     #Number of thiocarbonyl
     fr_Imine(mol),
     #Number of Imines
     fr_NH0(mol),
     #Number of Tertiary amines
     fr_NH1(mol),
     #Number of Secondary amines
     fr_NH2(mol),
     #Number of Primary amines
     fr_N_O(mol),
     #Number of hydroxylamine groups
     fr_Ndealkylation2(mol),
     #Number of tert-alicyclic amines (no heteroatoms, not quinine-like bridged N)
     fr_Nhpyrrole(mol),
     #Number of H-pyrrole nitrogens
     fr_SH(mol),
     #Number of thiol groups
     fr_aldehyde(mol),
     #Number of aldehydes
     fr_alkyl_carbamate(mol),
     #Number of alkyl carbamates (subject to hydrolysis)
     fr_alkyl_halide(mol),
     #Number of alkyl halides
     fr_allylic_oxid(mol),
     #Number of allylic oxidation sites excluding steroid dienone
     fr_amide(mol),
     #Number of amides
     fr_aniline(mol),
     #Number of anilines
     fr_aryl_methyl(mol),
     #Number of aryl methyl sites for hydroxylation
     fr_azo(mol),
     #Number of azo groups
     fr_benzene(mol),
     #Number of benzene rings
     fr_bicyclic(mol),
     #Bicyclic
     fr_dihydropyridine(mol),
     #Number of dihydropyridines
     fr_epoxide(mol),
     #Number of epoxide rings
     fr_ester(mol),
     #Number of esters
     fr_ether(mol),
     #Number of ether oxygens (including phenoxy)
     fr_furan(mol),
     #Number of furan rings
     fr_guanido(mol),
     #Number of guanidine groups
     fr_halogen(mol),
     #Number of halogens
     fr_imidazole(mol),
     #Number of imidazole rings
     fr_isothiocyan(mol),
     #Number of isothiocyanates
     fr_ketone(mol),
     #Number of ketones
     fr_ketone_Topliss(mol),
     #Number of ketones excluding diaryl, a,b-unsat. dienones, heteroatom on Calpha
     fr_lactam(mol),
     #Number of beta lactams
     fr_lactone(mol),
     #Number of cyclic esters (lactones)
     fr_methoxy(mol),
     #Number of methoxy groups -OCH3
     fr_nitrile(mol),
     #Number of nitriles
     fr_nitro(mol),
     #Number of nitro groups
     fr_oxazole(mol),
     #Number of oxazole rings
     fr_para_hydroxylation(mol),
     #Number of para-hydroxylation sites
     fr_phenol(mol),
     #Number of phenols
     fr_phos_acid(mol),
     #Number of phosphoric acid groups
     fr_phos_ester(mol),
     #Number of phosphoric ester groups
     fr_piperdine(mol),
     #Number of piperdine rings
     fr_priamide(mol),
     #Number of primary amides
     fr_pyridine(mol),
     #Number of pyridine rings
     fr_quatN(mol),
     #Number of quaternary nitrogens
     fr_sulfide(mol),
     #Number of thioether
     fr_thiazole(mol),
     #Number of thiazole rings
     fr_unbrch_alkane(mol)]
     #Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
    return gr

In [27]:
len(get_func_gr("OCC1(C(OC(CCCC)=O)C(OC(CCCCC)=O)C(COC(CCC)=O)O1)OC(O2)C(O)C(OC(CC)=O)C(O)C2COC(CC)=O"))

59

In [28]:
data_df['groups'] = data_df.apply(lambda x: get_func_gr(x.smiles_preprocessed), axis=1)
data_df

,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,num_of_sugars,groups
spectrum_id,,,,,,,,,,
(+)-Chelidonine_plasma_1338,[M+H]+,354.133599,"[135.045, 159.04369, 163.04027, 173.0605, 179....","[0.053, 0.011, 0.027, 0.015, 0.011, 0.034, 0.1...","[18.014429199999995, 30.038839199999984, 31.04...","[0.02, 0.045, 0.166, 0.026, 0.146, 0.041, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
(+)-Chelidonine_plasma_1340,[M+H]+,354.133599,"[275.0733, 305.08203, 323.09119, 324.09433, 35...","[0.017, 0.022, 0.028, 0.01, 1.0, 0.029]","[30.03926919999998, 31.04240920000001, 49.0515...","[0.01, 0.028, 0.022, 0.017]",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
(+)-Chelidonine_plasma_1343,[M+H]+,354.133599,"[135.04521, 143.05431, 159.04745, 161.0602, 16...","[0.101, 0.011, 0.012, 0.035, 0.081, 0.013, 0.0...","[18.008909200000005, 31.031639199999972, 31.04...","[0.016, 0.02, 0.012, 0.013, 0.028, 0.023, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
(+)-Chelidonine_plasma_1346,[M+H]+,354.133599,"[103.05478, 118.06499, 131.04845, 135.04384, 1...","[0.016, 0.01, 0.027, 0.034, 0.01, 0.016, 0.01,...","[79.06267919999999, 106.06514919999998, 107.05...","[0.023, 0.02, 0.063, 0.065, 0.038, 0.014, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
(+)-Chelidonine_plasma_1349,[M+H]+,354.133599,"[135.03654, 135.04454, 159.04517, 163.03651, 1...","[0.01, 0.047, 0.011, 0.016, 0.017, 0.014, 0.11...","[18.00828919999998, 30.03291919999998, 30.0438...","[0.028, 0.025, 0.015, 0.148, 0.03, 0.126, 0.04...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
...,...,...,...,...,...,...,...,...,...,...
trans-pterostilbene_plasma_3538,[M+H]+,257.117221,"[55.01796, 77.03296, 77.03867, 79.02107, 79.05...","[0.021, 0.021, 0.046, 0.011, 0.046, 0.02, 0.01...","[15.022540799999973, 16.0322008, 17.0336207999...","[0.259, 0.209, 0.02, 0.017, 0.034, 0.038, 0.01...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
trans-pterostilbene_plasma_3540,[M+H]+,257.117221,"[65.03999, 75.02646, 77.02925, 77.03927, 78.04...","[0.057, 0.011, 0.011, 0.115, 0.013, 0.043, 0.0...","[31.055850799999973, 45.04376079999997, 46.038...","[0.026, 0.011, 0.012, 0.013, 0.037, 0.016, 0.0...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
trans-pterostilbene_plasma_3542,[M+H]+,257.117221,"[91.05325, 103.0554, 105.07047, 107.05009, 111...","[0.015, 0.01, 0.058, 0.021, 0.034, 0.02, 0.015...","[15.01973079999999, 31.012410799999998, 31.021...","[0.043, 0.016, 0.018, 0.046, 0.012, 0.03, 0.03...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [29]:
funct=["Number of aliphatic carboxylic acids",
 "Number of aliphatic hydroxyl groups",
 "Number of aliphatic hydroxyl groups excluding tert-OH",
 "Number of N functional groups attached to aromatics",
 "Number of Aromatic carboxylic acides",
 "Number of aromatic nitrogens",
 "Number of aromatic amines",
 "Number of aromatic hydroxyl groups",
 "Number of carboxylic acids",
 "Number of carboxylic acids",
 "Number of carbonyl O",
 "Number of carbonyl O, excluding COOH",
 "Number of thiocarbonyl",
 "Number of Imines",
 "Number of Tertiary amines",
 "Number of Secondary amines",
 "Number of Primary amines",
 "Number of hydroxylamine groups",
 "Number of tert-alicyclic amines (no heteroatoms, not quinine-like bridged N)",
 "Number of H-pyrrole nitrogens",
 "Number of thiol groups",
 "Number of aldehydes",
 "Number of alkyl carbamates (subject to hydrolysis)",
 "Number of alkyl halides",
 "Number of allylic oxidation sites excluding steroid dienone",
 "Number of amides",
 "Number of anilines",
 "Number of aryl methyl sites for hydroxylation",
 "Number of azo groups",
 "Number of benzene rings",
 "Bicyclic",
 "Number of dihydropyridines",
 "Number of epoxide rings",
 "Number of esters",
 "Number of ether oxygens (including phenoxy)",
 "Number of furan rings",
 "Number of guanidine groups",
 "Number of halogens",
 "Number of imidazole rings",
 "Number of isothiocyanates",
 "Number of ketones",
 "Number of ketones excluding diaryl, a,b-unsat. dienones, heteroatom on Calpha",
 "Number of beta lactams",
 "Number of cyclic esters (lactones)",
 "Number of methoxy groups -OCH3",
 "Number of nitriles",
 "Number of nitro groups",
 "Number of oxazole rings",
 "Number of para-hydroxylation sites",
 "Number of phenols",
 "Number of phosphoric acid groups",
 "Number of phosphoric ester groups",
 "Number of piperdine rings",
 "Number of primary amides",
 "Number of pyridine rings",
 "Number of quaternary nitrogens",
 "Number of thioether",
 "Number of thiazole rings",
 "Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)"]

In [30]:
groups_df = pd.DataFrame(data_df['groups'].to_list(), columns=funct)
groups_df

,Number of aliphatic carboxylic acids,Number of aliphatic hydroxyl groups,Number of aliphatic hydroxyl groups excluding tert-OH,Number of N functional groups attached to aromatics,Number of Aromatic carboxylic acides,Number of aromatic nitrogens,Number of aromatic amines,Number of aromatic hydroxyl groups,Number of carboxylic acids,Number of carboxylic acids,...,Number of phenols,Number of phosphoric acid groups,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83353,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
83354,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
83355,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
83356,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [31]:
data_df=data_df.reset_index()
data_df

,spectrum_id,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,num_of_sugars,groups
0,(+)-Chelidonine_plasma_1338,[M+H]+,354.133599,"[135.045, 159.04369, 163.04027, 173.0605, 179....","[0.053, 0.011, 0.027, 0.015, 0.011, 0.034, 0.1...","[18.014429199999995, 30.038839199999984, 31.04...","[0.02, 0.045, 0.166, 0.026, 0.146, 0.041, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
1,(+)-Chelidonine_plasma_1340,[M+H]+,354.133599,"[275.0733, 305.08203, 323.09119, 324.09433, 35...","[0.017, 0.022, 0.028, 0.01, 1.0, 0.029]","[30.03926919999998, 31.04240920000001, 49.0515...","[0.01, 0.028, 0.022, 0.017]",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
2,(+)-Chelidonine_plasma_1343,[M+H]+,354.133599,"[135.04521, 143.05431, 159.04745, 161.0602, 16...","[0.101, 0.011, 0.012, 0.035, 0.081, 0.013, 0.0...","[18.008909200000005, 31.031639199999972, 31.04...","[0.016, 0.02, 0.012, 0.013, 0.028, 0.023, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,(+)-Chelidonine_plasma_1346,[M+H]+,354.133599,"[103.05478, 118.06499, 131.04845, 135.04384, 1...","[0.016, 0.01, 0.027, 0.034, 0.01, 0.016, 0.01,...","[79.06267919999999, 106.06514919999998, 107.05...","[0.023, 0.02, 0.063, 0.065, 0.038, 0.014, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,(+)-Chelidonine_plasma_1349,[M+H]+,354.133599,"[135.03654, 135.04454, 159.04517, 163.03651, 1...","[0.01, 0.047, 0.011, 0.016, 0.017, 0.014, 0.11...","[18.00828919999998, 30.03291919999998, 30.0438...","[0.028, 0.025, 0.015, 0.148, 0.03, 0.126, 0.04...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...
83353,trans-pterostilbene_plasma_3538,[M+H]+,257.117221,"[55.01796, 77.03296, 77.03867, 79.02107, 79.05...","[0.021, 0.021, 0.046, 0.011, 0.046, 0.02, 0.01...","[15.022540799999973, 16.0322008, 17.0336207999...","[0.259, 0.209, 0.02, 0.017, 0.034, 0.038, 0.01...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
83354,trans-pterostilbene_plasma_3540,[M+H]+,257.117221,"[65.03999, 75.02646, 77.02925, 77.03927, 78.04...","[0.057, 0.011, 0.011, 0.115, 0.013, 0.043, 0.0...","[31.055850799999973, 45.04376079999997, 46.038...","[0.026, 0.011, 0.012, 0.013, 0.037, 0.016, 0.0...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
83355,trans-pterostilbene_plasma_3542,[M+H]+,257.117221,"[91.05325, 103.0554, 105.07047, 107.05009, 111...","[0.015, 0.01, 0.058, 0.021, 0.034, 0.02, 0.015...","[15.01973079999999, 31.012410799999998, 31.021...","[0.043, 0.016, 0.018, 0.046, 0.012, 0.03, 0.03...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
83356,trans-pterostilbene_plasma_3544,[M+H]+,257.117221,"[55.02081, 68.99757, 77.03687, 77.04144, 77.04...","[0.011, 0.01, 0.021, 0.018, 0.011, 0.013, 0.03...","[15.023320799999993, 16.02811079999998, 16.044...","[0.231, 0.163, 0.02, 0.01, 0.225, 0.011, 0.014...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [32]:
data_df=data_df.join(groups_df,how="outer")
data_df

,spectrum_id,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,num_of_sugars,...,Number of phenols,Number of phosphoric acid groups,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
0,(+)-Chelidonine_plasma_1338,[M+H]+,354.133599,"[135.045, 159.04369, 163.04027, 173.0605, 179....","[0.053, 0.011, 0.027, 0.015, 0.011, 0.034, 0.1...","[18.014429199999995, 30.038839199999984, 31.04...","[0.02, 0.045, 0.166, 0.026, 0.146, 0.041, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
1,(+)-Chelidonine_plasma_1340,[M+H]+,354.133599,"[275.0733, 305.08203, 323.09119, 324.09433, 35...","[0.017, 0.022, 0.028, 0.01, 1.0, 0.029]","[30.03926919999998, 31.04240920000001, 49.0515...","[0.01, 0.028, 0.022, 0.017]",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
2,(+)-Chelidonine_plasma_1343,[M+H]+,354.133599,"[135.04521, 143.05431, 159.04745, 161.0602, 16...","[0.101, 0.011, 0.012, 0.035, 0.081, 0.013, 0.0...","[18.008909200000005, 31.031639199999972, 31.04...","[0.016, 0.02, 0.012, 0.013, 0.028, 0.023, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
3,(+)-Chelidonine_plasma_1346,[M+H]+,354.133599,"[103.05478, 118.06499, 131.04845, 135.04384, 1...","[0.016, 0.01, 0.027, 0.034, 0.01, 0.016, 0.01,...","[79.06267919999999, 106.06514919999998, 107.05...","[0.023, 0.02, 0.063, 0.065, 0.038, 0.014, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
4,(+)-Chelidonine_plasma_1349,[M+H]+,354.133599,"[135.03654, 135.04454, 159.04517, 163.03651, 1...","[0.01, 0.047, 0.011, 0.016, 0.017, 0.014, 0.11...","[18.00828919999998, 30.03291919999998, 30.0438...","[0.028, 0.025, 0.015, 0.148, 0.03, 0.126, 0.04...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83353,trans-pterostilbene_plasma_3538,[M+H]+,257.117221,"[55.01796, 77.03296, 77.03867, 79.02107, 79.05...","[0.021, 0.021, 0.046, 0.011, 0.046, 0.02, 0.01...","[15.022540799999973, 16.0322008, 17.0336207999...","[0.259, 0.209, 0.02, 0.017, 0.034, 0.038, 0.01...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,...,1,0,0,0,0,0,0,0,0,0
83354,trans-pterostilbene_plasma_3540,[M+H]+,257.117221,"[65.03999, 75.02646, 77.02925, 77.03927, 78.04...","[0.057, 0.011, 0.011, 0.115, 0.013, 0.043, 0.0...","[31.055850799999973, 45.04376079999997, 46.038...","[0.026, 0.011, 0.012, 0.013, 0.037, 0.016, 0.0...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,...,1,0,0,0,0,0,0,0,0,0
83355,trans-pterostilbene_plasma_3542,[M+H]+,257.117221,"[91.05325, 103.0554, 105.07047, 107.05009, 111...","[0.015, 0.01, 0.058, 0.021, 0.034, 0.02, 0.015...","[15.01973079999999, 31.012410799999998, 31.021...","[0.043, 0.016, 0.018, 0.046, 0.012, 0.03, 0.03...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,...,1,0,0,0,0,0,0,0,0,0
83356,trans-pterostilbene_plasma_3544,[M+H]+,257.117221,"[55.02081, 68.99757, 77.03687, 77.04144, 77.04...","[0.011, 0.01, 0.021, 0.018, 0.011, 0.013, 0.03...","[15.023320799999993, 16.02811079999998, 16.044...","[0.231, 0.163, 0.02, 0.01, 0.225, 0.011, 0.014...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,...,1,0,0,0,0,0,0,0,0,0


In [33]:
data_df.iloc[:, 9]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
83353    0.0
83354    0.0
83355    0.0
83356    0.0
83357    0.0
Name: num_of_sugars, Length: 83358, dtype: float64

In [34]:
data_df=data_df.drop(["groups"], axis=1)
data_df

,spectrum_id,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,num_of_sugars,...,Number of phenols,Number of phosphoric acid groups,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
0,(+)-Chelidonine_plasma_1338,[M+H]+,354.133599,"[135.045, 159.04369, 163.04027, 173.0605, 179....","[0.053, 0.011, 0.027, 0.015, 0.011, 0.034, 0.1...","[18.014429199999995, 30.038839199999984, 31.04...","[0.02, 0.045, 0.166, 0.026, 0.146, 0.041, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
1,(+)-Chelidonine_plasma_1340,[M+H]+,354.133599,"[275.0733, 305.08203, 323.09119, 324.09433, 35...","[0.017, 0.022, 0.028, 0.01, 1.0, 0.029]","[30.03926919999998, 31.04240920000001, 49.0515...","[0.01, 0.028, 0.022, 0.017]",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
2,(+)-Chelidonine_plasma_1343,[M+H]+,354.133599,"[135.04521, 143.05431, 159.04745, 161.0602, 16...","[0.101, 0.011, 0.012, 0.035, 0.081, 0.013, 0.0...","[18.008909200000005, 31.031639199999972, 31.04...","[0.016, 0.02, 0.012, 0.013, 0.028, 0.023, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
3,(+)-Chelidonine_plasma_1346,[M+H]+,354.133599,"[103.05478, 118.06499, 131.04845, 135.04384, 1...","[0.016, 0.01, 0.027, 0.034, 0.01, 0.016, 0.01,...","[79.06267919999999, 106.06514919999998, 107.05...","[0.023, 0.02, 0.063, 0.065, 0.038, 0.014, 0.01...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
4,(+)-Chelidonine_plasma_1349,[M+H]+,354.133599,"[135.03654, 135.04454, 159.04517, 163.03651, 1...","[0.01, 0.047, 0.011, 0.016, 0.017, 0.014, 0.11...","[18.00828919999998, 30.03291919999998, 30.0438...","[0.028, 0.025, 0.015, 0.148, 0.03, 0.126, 0.04...",GHKISGDRQRSCII-UHFFFAOYNA-N,CN1Cc2c(ccc3c2OCO3)C2C(O)Cc3cc4c(cc3C21)OCO4,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83353,trans-pterostilbene_plasma_3538,[M+H]+,257.117221,"[55.01796, 77.03296, 77.03867, 79.02107, 79.05...","[0.021, 0.021, 0.046, 0.011, 0.046, 0.02, 0.01...","[15.022540799999973, 16.0322008, 17.0336207999...","[0.259, 0.209, 0.02, 0.017, 0.034, 0.038, 0.01...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,...,1,0,0,0,0,0,0,0,0,0
83354,trans-pterostilbene_plasma_3540,[M+H]+,257.117221,"[65.03999, 75.02646, 77.02925, 77.03927, 78.04...","[0.057, 0.011, 0.011, 0.115, 0.013, 0.043, 0.0...","[31.055850799999973, 45.04376079999997, 46.038...","[0.026, 0.011, 0.012, 0.013, 0.037, 0.016, 0.0...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,...,1,0,0,0,0,0,0,0,0,0
83355,trans-pterostilbene_plasma_3542,[M+H]+,257.117221,"[91.05325, 103.0554, 105.07047, 107.05009, 111...","[0.015, 0.01, 0.058, 0.021, 0.034, 0.02, 0.015...","[15.01973079999999, 31.012410799999998, 31.021...","[0.043, 0.016, 0.018, 0.046, 0.012, 0.03, 0.03...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,...,1,0,0,0,0,0,0,0,0,0
83356,trans-pterostilbene_plasma_3544,[M+H]+,257.117221,"[55.02081, 68.99757, 77.03687, 77.04144, 77.04...","[0.011, 0.01, 0.021, 0.018, 0.011, 0.013, 0.03...","[15.023320799999993, 16.02811079999998, 16.044...","[0.231, 0.163, 0.02, 0.01, 0.225, 0.011, 0.014...",VLEUZFDZJKSGMX-ONEGZZNKSA-N,COc1cc(C=Cc2ccc(O)cc2)cc(OC)c1,0.0,...,1,0,0,0,0,0,0,0,0,0


In [35]:
group_array=data_df.iloc[:, 9:]


In [36]:
group_array.to_numpy().max()

23.0

In [37]:
group_array.to_numpy().shape

(83358, 60)

In [38]:
train_int=group_array.to_numpy()

In [39]:

def make_float_targets(k):
  targets = np.zeros(k, dtype=np.float32)
  start = 1.0 / (2 * k)  # like 0.125
  delta = 1.0 / k        # like 0.250
  for i in range(k):
    targets[i] = start + (i * delta) 
  return targets

In [40]:
make_float_targets(24)


array([0.02083333, 0.0625    , 0.10416666, 0.14583333, 0.1875    ,
       0.22916667, 0.27083334, 0.3125    , 0.35416666, 0.39583334,
       0.4375    , 0.47916666, 0.5208333 , 0.5625    , 0.6041667 ,
       0.6458333 , 0.6875    , 0.7291667 , 0.7708333 , 0.8125    ,
       0.8541667 , 0.8958333 , 0.9375    , 0.9791667 ], dtype=float32)

In [38]:
ordinal=make_float_targets(24)

In [42]:
ordinal[2]

0.104166664

#data_df = pd.read_csv("/Users/delser/mass2smiles/retrain/nist/tcn_seq/all_HRMS_train_17012023_cddd_refine_s.tsv",sep="\t")
data_df = pd.read_csv("/Users/delser/mass2smiles/retrain/nist/tcn_seq/all_HRMS_valid_17012023_cddd_refine.tsv",sep="\t")
#data_df = pd.read_csv("/Users/delser/mass2smiles/retrain/nist/all_HRMS_validation_16122022_cddd_refine.tsv",sep="\t")
data_df=data_df.dropna()
data_df

In [41]:
data_df = data_df.sample(frac = 1)
data_df

,spectrum_id,adduct,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,inchikey,smiles_preprocessed,num_of_sugars,...,Number of phenols,Number of phosphoric acid groups,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
24219,CCMSLIB00005726986,[M+H]+,342.07700,"[53.002701, 69.069901, 70.039902, 122.999496, ...","[0.010063446472489251, 0.09171604559744392, 0....","[151.11110299999999, 169.110798, 169.121601, 1...","[0.014194703372154123, 0.014742377244715292, 0...",STJLVHWMYQXCPB-UHFFFAOYSA-N,CCCC1COC(Cn2cncn2)(c2ccc(Cl)cc2Cl)O1,0.0,...,0,0,0,0,0,0,0,0,0,0
36520,CCMSLIB00006115565,[M+H]+,466.31700,"[55.0541, 60.922901, 67.054001, 70.752098, 71....","[0.16146962465733394, 0.06493402643554223, 0.0...","[54.03019599999999, 80.65830900000003, 129.063...","[0.28292792134215433, 0.08448161553784861, 0.3...",RFDAIACWWDREDC-FRVQLJSFSA-N,CC(CCC(=O)NCC(=O)O)C1CCC2C3C(O)CC4CC(O)CCC4(C)...,0.0,...,0,0,0,0,0,0,0,0,0,0
57525,NIST114555,[M+H]+,420.96970,"[50.0148, 51.0227, 53.002, 53.0383, 55.0175, 5...","[0.032002002002002, 0.07480480480480481, 0.011...","[18.01139999999998, 32.03190000000001, 36.0030...","[1.0, 1.0, 0.0533033033033033, 0.0147047047047...",GKDKOMAJZATYAY-XCMZKKERSA-N,O=P(O)(O)OC1C(O)C(O)C(O)C(OP(=O)(O)O)C1OP(=O)(O)O,0.0,...,0,3,3,0,0,0,0,0,0,0
18453,CCMSLIB00004714529,[M+H]+,485.31100,"[53.038925, 55.018158, 55.05455, 56.049793, 56...","[0.056912830000000005, 0.02006357, 0.355427699...","[244.40157599999998, 246.13166299999997, 258.1...","[0.024042059999999997, 0.03417382, 0.02122381,...",JECOCDKQCZSANH-RFKAOMIISA-N,C=C1C(OC2OC(CO)C(O)C(O)C2O)CC2C(C)(C)C(O)CCC2(...,1.0,...,0,0,0,0,0,0,0,0,0,0
57589,NIST116969,[M+H]+,311.15540,"[50.0149, 51.0227, 54.0336, 57.0132, 58.065, 5...","[0.023103103103103102, 0.0355955955955956, 0.0...","[17.007499999999993, 18.010699999999986, 31.04...","[0.035005005005005, 1.0, 0.2139039039039039, 0...",PTJADDMMFYXMMG-IBGZPJMESA-N,CNCCCC1(c2ccc(F)cc2)OCc2cc(C#N)ccc21,0.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42002,CCMSLIB00006673970,[M-H2O+H]+,120.04400,"[50.716675, 52.040119, 54.318787, 54.323826, 5...","[0.024994948916947344, 0.02298565382566748, 0....","[16.09871799999999, 22.15638899999999, 22.1781...","[0.025422772046311003, 0.027329820729600093, 0...",RWZYAGGXGHYGMB-UHFFFAOYSA-N,Nc1ccccc1C(=O)O,0.0,...,0,0,0,0,0,0,0,0,0,0
77441,NIST900303,[M+H]+,174.05500,"[50.0153, 51.0231, 52.0184, 53.0388, 61.0075, ...","[0.2785985985985986, 1.0, 0.016596596596596593...","[27.994699999999995, 44.01480000000001, 45.010...","[1.0, 0.07220220220220219, 0.4573973973973974,...",KVMMIDQDXZOPAB-UHFFFAOYSA-N,O=C(O)c1cc2ccccc2cn1,0.0,...,0,0,0,0,0,1,0,0,0,0
54204,DB#=MSBNK-Stacey_Reinke_ECU-Stacey_Reinke_pos_...,[M+H]+,133.09715,"[53.0028, 56.3404, 59.9308, 73.1201, 114.372, ...","[1.0, 0.644, 0.6709999999999999, 0.738, 0.65, ...","[18.72515, 59.977050000000006, 73.16635, 76.75...","[0.65, 0.738, 0.6709999999999999, 0.644, 1.0]",AHLPHDHHMVZTML-UHFFFAOYSA-N,NCCCC(N)C(=O)O,0.0,...,0,0,0,0,0,0,0,0,0,0
63496,NIST323648,[M+H]+,384.99490,"[51.9398, 55.9342, 59.9663, 62.945, 63.0262, 6...","[0.0365965965965966, 0.0662962962962963, 0.024...","[34.12209999999999, 35.12299999999999, 45.1438...","[0.017197197197197197, 0.027797797797797796, 0...",RIZMRRKBZQXFOY-UHFFFAOYSA-N,CCOP(=S)(OCC)SCSP(=S)(OCC)OCC,0.0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
data_df.to_csv('/Users/delser/mass2smiles/retrain/nist/tcn_seq/all_HRMS_train_24012023_cddd_refine_s.tsv',sep='\t')

In [43]:
#data_df=data_df.drop(["Unnamed: 0"], axis=1)

group_array=data_df.iloc[:, 9:]


In [44]:
train_int=group_array.to_numpy()
ordinal=make_float_targets(24)
all_arr=[]
for sample in train_int:
    cache=[]
    for i in sample:
        cache.append(ordinal[int(i)])
    x=np.array(cache)
    all_arr.append(x)
    
train_y=np.stack(all_arr)   
train_y.shape  

(83358, 60)

In [45]:
np.save('y1_all_HRMS_train_24012023_cddd.npy',train_y)